# Data Normalization


In [ ]:
!pip install scikit-learn
!pip install scikeras
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pandas as pd
import os

In [ ]:
target_directory = "/content/drive/MyDrive/TLSHandshakesTraffic/"
directories = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
# Print all devices:
print("List of devices traffic to pre-process:")

In [ ]:
for directory in directories:
  print(directory)
  cl_file = target_directory+directory+'/ClientHello-new.csv'
  existing_df = pd.read_csv(cl_file, header=None, delimiter=';')
  if 5 in existing_df.columns:
    pass
  else:
    existing_df[5] = 0
  for index, row in existing_df.iterrows():
    try:
        value = float(row[4])
        existing_df.at[index, 5] = value
        existing_df.at[index, 4] = 0
    except ValueError:
        pass
  sh_file = cl_file = target_directory+directory+'/ServerHello-new.csv'
  sh_pd = pd.read_csv(cl_file, header=None, delimiter=';')
  sh_len = len(sh_pd)
  ch_len = len(existing_df)

  next_column_number = len(existing_df.columns)
  existing_df[next_column_number] = sh_pd[1].iloc[:ch_len % sh_len]
  next_column_number = len(existing_df.columns)
  existing_df[next_column_number] = sh_pd[2].iloc[:ch_len % sh_len]


  new_column = pd.Series([directory] * len(existing_df))
  next_column_number = len(existing_df.columns)
  print(existing_df)


  existing_df[next_column_number] = new_column
  existing_df.to_csv(target_directory+'features.csv', mode='a', header=False, index=False, sep=';')
print("Rows appended successfully!")

In [ ]:
from sklearn.model_selection import train_test_split
files = '/content/drive/MyDrive/TLSHandshakesTraffic/features.csv'
datass = pd.read_csv(files, header=None, delimiter=';')
shuffled_data = datass.sample(frac=1, random_state=42)
total_rows = len(shuffled_data)
train_rows = int(0.7 * total_rows)

train = shuffled_data[train_rows]
test = shuffled_data[train_rows:]

train.to_csv('/content/drive/MyDrive/TLSHandshakesTraffic/train_f.csv', header=None, index=False, sep=';')
#test.to_csv('/content/drive/MyDrive/TLSHandshakesTraffic/test_f.csv', header=None, index=False, sep=';')


In [ ]:
from ctypes import *

def hex_to_float(hex_string):
    i = int(hex_string, 16)  # Convert from hex to a Python integer
    cp = pointer(c_int(i))   # Create a C integer pointer
    fp = cast(cp, POINTER(c_float))  # Cast the int pointer to a float pointer
    return fp.contents.value  # Dereference the pointer to get the float value

# pre train


In [ ]:
!pip install joblib
!pip install optuna
import re

def extract_server_name(server_string):
    if isinstance(server_string, str):
        pattern = r"servername='(.*?)'"
        server_names = re.findall(pattern, server_string)
        return server_names[0] if server_names else None
    elif hasattr(server_string, 'servername'):
        return server_string.servername
    else:
        return None

In [ ]:

import os
import pandas as pd
target_directory = "/content/drive/MyDrive/TLSHandshakesTraffic/"
directories = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
i=0
for directory in directories:

    print(str(i)+' '+directory)
    i = i + 1
from sklearn.model_selection import train_test_split
file_train = '/content/drive/MyDrive/TLSHandshakesTraffic/features.csv'
train_data = pd.read_csv(file_train, header=None, delimiter=';')

y = train_data[8]
print(train_data[4])

# set the lenghts as same
#train_data[2] = 64
#train_data[3]=64
#train_data[0]=64
#train_data[5]=64

train_data[1] = train_data[1].apply(lambda x: int(x, 16))
train_data[4] = train_data[4].apply(lambda x: extract_server_name(x))

unique_strings = train_data[4].unique()


string_to_numeric = {}

# Assign numeric representations to string values starting from 1
for i, string in enumerate(unique_strings, start=1):
    string_to_numeric[string] = i
train_data[4] = train_data[4].map(string_to_numeric)
train_data.fillna(0, inplace=True)
train_data.replace('None', 0, inplace=True)
train_data.replace(' ', 0, inplace=True)
X = train_data.drop(columns=[8,6,4])
X.columns = range(X.shape[1])

from sklearn.preprocessing import LabelEncoder

label_mapping = {device_name: label for label, device_name in enumerate(directories)}

# assistant=1 Kitchen=2 Hub=3 Plug=4 Door=5 Smarttv= 6 Camera=7
table = {
    '18_74_2e_41_4d_35': 1,
    '20_df_b9_5f_41_7e': 1,
    '20_f8_5e_cc_18_1f': 2,
    '0_71_47_c0_91_93': 1,
    '0_e_f3_3b_85_e5': 3,
    '0_21_cc_4d_ce_8c': 3,
    '18_b4_30_c8_d8_28':4,
    '0_17_88_68_5f_61': 3,
    '14_91_82_b4_4b_5f': 4,
    '0_3_7f_4f_c6_b5': 3,
    '68_db_f5_90_19_64': 1,
    '34_29_8f_1e_41_25': 5,
    '7c_c7_9_56_6e_48': 5,
    '50_c7_bf_5a_2e_a0': 4,
    '50_c7_bf_a0_f3_76': 4,
    '38_8c_50_68_d7_5c': 6,
    '24_fd_5b_4_1b_75': 3,
    '6c_56_97_35_39_f4': 6,
    '6c_72_20_c5_a_3f': 6,
    '70_2c_1f_3b_36_53':2,
    '8_66_98_a2_21_9e': 6,
    '98_84_e3_e4_35_bd': 5,
    'b0_d5_9d_b9_f0_b4': 7,
    '84_c0_ef_2f_42_cc': 6,
    '88_de_a9_8_3_b9': 6,
    'b0_ce_18_27_9f_e4': 3,
    'b4_e6_2a_27_34_b7': 2,
    'bc_dd_c2_38_d_32': 4,
    '9c_8e_cd_a_33_1b': 7,
    'b0_fc_d_c9_0_4c': 7,
    'c0_97_27_81_67_99': 2,
    'd4_a3_3d_6b_1e_97': 1,
    'd8_f7_10_c3_34_e4': 1,
    'c0_97_27_73_aa_38': 2,
    'fc_a1_83_38_e0_2d': 1,
    'ec_fa_bc_2e_81_f7':3,
    'd8_28_c9_10_b5_60': 2,
    'dc_4f_22_5b_1a_a3': 4,
    'f4_b8_5e_31_73_db': 7,
}


tablefabric = {
    '18_74_2e_41_4d_35': 1,
    '20_df_b9_5f_41_7e': 2,
    '20_f8_5e_cc_18_1f': 3,
    '0_71_47_c0_91_93': 1,
    '0_e_f3_3b_85_e5': 4,
    '0_21_cc_4d_ce_8c': 5,
    '18_b4_30_c8_d8_28':6,
    '0_17_88_68_5f_61': 7,
    '14_91_82_b4_4b_5f': 8,
    '0_3_7f_4f_c6_b5': 9,
    '68_db_f5_90_19_64': 1,
    '34_29_8f_1e_41_25': 10,
    '7c_c7_9_56_6e_48': 11,
    '50_c7_bf_5a_2e_a0': 12,
    '50_c7_bf_a0_f3_76': 12,
    '38_8c_50_68_d7_5c': 13,
    '24_fd_5b_4_1b_75': 14,
    '6c_56_97_35_39_f4': 1,
    '6c_72_20_c5_a_3f': 15,
    '70_2c_1f_3b_36_53':16,
    '8_66_98_a2_21_9e': 17,
    '98_84_e3_e4_35_bd': 18,
    'b0_d5_9d_b9_f0_b4': 19,
    '84_c0_ef_2f_42_cc': 20,
    '88_de_a9_8_3_b9': 21,
    'b0_ce_18_27_9f_e4': 22,
    'b4_e6_2a_27_34_b7': 13,
    'bc_dd_c2_38_d_32': 23,
    '9c_8e_cd_a_33_1b': 24,
    'b0_fc_d_c9_0_4c': 1,
    'c0_97_27_81_67_99': 20,
    'd4_a3_3d_6b_1e_97': 17,
    'd8_f7_10_c3_34_e4': 27,
    'c0_97_27_73_aa_38': 20,
    'fc_a1_83_38_e0_2d': 1,
    'ec_fa_bc_2e_81_f7':26,
    'd8_28_c9_10_b5_60': 27,
    'dc_4f_22_5b_1a_a3': 26,
    'f4_b8_5e_31_73_db': 28,
}


#By type of device
encoded_y = [table[name] for name in y]

#by manyfacturer
#encoded_y = [tablefabric[name] for name in y]

# For type of device or type of manufacturer
X_train, X_val, y_train, y_val = train_test_split(X, encoded_y, test_size=0.30, random_state=42)

#for device
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)

0 18_74_2e_41_4d_35
1 20_df_b9_5f_41_7e
2 20_f8_5e_cc_18_1f
3 0_71_47_c0_91_93
4 0_e_f3_3b_85_e5
5 0_21_cc_4d_ce_8c
6 18_b4_30_c8_d8_28
7 0_17_88_68_5f_61
8 14_91_82_b4_4b_5f
9 0_3_7f_4f_c6_b5
10 68_db_f5_90_19_64
11 34_29_8f_1e_41_25
12 7c_c7_9_56_6e_48
13 50_c7_bf_5a_2e_a0
14 50_c7_bf_a0_f3_76
15 38_8c_50_68_d7_5c
16 24_fd_5b_4_1b_75
17 6c_56_97_35_39_f4
18 6c_72_20_c5_a_3f
19 70_2c_1f_3b_36_53
20 8_66_98_a2_21_9e
21 98_84_e3_e4_35_bd
22 b0_d5_9d_b9_f0_b4
23 84_c0_ef_2f_42_cc
24 88_de_a9_8_3_b9
25 b0_ce_18_27_9f_e4
26 b4_e6_2a_27_34_b7
27 bc_dd_c2_38_d_32
28 9c_8e_cd_a_33_1b
29 b0_fc_d_c9_0_4c
30 c0_97_27_81_67_99
31 d4_a3_3d_6b_1e_97
32 d8_f7_10_c3_34_e4
33 c0_97_27_73_aa_38
34 fc_a1_83_38_e0_2d
35 ec_fa_bc_2e_81_f7
36 d8_28_c9_10_b5_60
37 dc_4f_22_5b_1a_a3
38 f4_b8_5e_31_73_db


<ipython-input-6-30da1d28cd38>:12: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(file_train, header=None, delimiter=';')


0          [<ServerName  nametype=host_name namelen=28 se...
1          [<ServerName  nametype=host_name namelen=28 se...
2          [<ServerName  nametype=host_name namelen=28 se...
3          [<ServerName  nametype=host_name namelen=28 se...
4          [<ServerName  nametype=host_name namelen=19 se...
                                 ...                        
3218237                                                    0
3218238                                                    0
3218239                                                    0
3218240                                                    0
3218241                                                    0
Name: 4, Length: 3218242, dtype: object


# SVM

Best parameters

In [ ]:
import optuna
import time
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

# Scale data
sc = StandardScaler()
X_tr = sc.fit_transform(X_train)
X_test = sc.transform(X_val)

def objective_svm(trial):
    # Hyperparameters
    C = trial.suggest_loguniform("C", 1e-3, 10)
    tol = trial.suggest_loguniform("tol", 1e-6, 1e-2)

    # Model
    svm = LinearSVC(random_state=0, tol=tol, C=C)
    clf = CalibratedClassifierCV(svm)
    model = OneVsRestClassifier(clf, n_jobs=-1)

    #train
    model.fit(X_tr, y_train)

    y_pred_svm = model.predict(X_test)

    return accuracy_score(y_val, y_pred_svm)

# Optuna
study_svm = optuna.create_study(direction="maximize")
study_svm.optimize(objective_svm, n_trials=3)

best_params_svm = study_svm.best_params
print("\n Best SVM Hyperparameters:", best_params_svm)

Training

In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

best_C = 0.23218097496132672
best_tol = 2.2313982919324327e-05

sc = StandardScaler()
X_tr = sc.fit_transform(X_train)
X_test = sc.transform(X_val)

svm = LinearSVC(random_state=0, tol=best_tol, C=best_C)
clf = CalibratedClassifierCV(svm)
model = OneVsRestClassifier(clf, n_jobs=-1)

model.fit(X_tr, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average="weighted", zero_division=0)
recall = recall_score(y_val, y_pred, average="weighted", zero_division=0)
f1 = f1_score(y_val, y_pred, average="weighted", zero_division=0)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")



# Deep learning

Best parameters

In [ ]:
import optuna
import time
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import accuracy_score

sc = StandardScaler()
X_train2 = sc.fit_transform(X_train)
X_test = sc.transform(X_val)

y_train_bin = label_binarize(y_train, classes=np.unique(y_train))
y_val_bin = label_binarize(y_val, classes=np.unique(y_train))

def objective_nn(trial):
    # Hyperparameters
    num_units = trial.suggest_int("units", 16, 128, step=16)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    epochs = trial.suggest_int("epochs", 10, 50, step=5)

    # Model
    model = Sequential([
        Dense(num_units, activation='relu', input_dim=X_train2.shape[1]),
        Dense(num_units, activation='relu'),
        Dense(y_train_bin.shape[1], activation='softmax')
    ])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(X_train2, y_train_bin, epochs=epochs, batch_size=batch_size, verbose=0)

    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_val_true = np.argmax(y_val_bin, axis=1)

    return accuracy_score(y_val_true, y_pred)

# Optuna
study_nn = optuna.create_study(direction="maximize")
study_nn.optimize(objective_nn, n_trials=3)

best_params_nn = study_nn.best_params
print("\n Best Deep learning Hyperparameters:", best_params_nn)


Training

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import numpy
import time
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize

# changes depending on what is classifying, if tyoe of device or device manufacturer or device
sc = StandardScaler()
X_train2 = sc.fit_transform(X_train)
X_test = sc.transform(X_val)

y_train_bin = label_binarize(y_train, classes=np.unique(y_train))
y_val_bin = label_binarize(y_val, classes=np.unique(y_train))

model = Sequential()

model.add(Dense(64, activation='relu', input_dim=6))
model.add(Dense(64, activation='relu'))
model.add(Dense(27, activation='softmax'))

optimizer = Adam(learning_rate=0.0004274800244154512)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

start_time = time.time()
model.fit(X_train2, y_train_bin, epochs=10, batch_size=64)
elapsed_time = time.time() - start_time
print(f"Training time: done {elapsed_time:.2f} seconds")

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score,f1_score
start_time = time.time()
y_pred_prob = model.predict(X_test)
y_pred_bin = (y_pred_prob > 0.5).astype(int)
accuracy = accuracy_score(y_val_bin, y_pred_bin)
precision = precision_score(y_val_bin, y_pred_bin, average='micro')
recall = recall_score(y_val_bin, y_pred_bin, average='micro')
f1 = f1_score(y_val_bin, y_pred_bin, average='micro')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


# Random Forest

Best params

In [ ]:
import optuna
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score

sc = RobustScaler()
X_train2 = sc.fit_transform(X_train)
X_test = sc.transform(X_val)

def objective_rf(trial):

    n_estimators = trial.suggest_int("n_estimators", 50, 300, step=50)
    max_depth = trial.suggest_int("max_depth", 5, 50, step=5)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=1,
        n_jobs=-1
    )
    clf.fit(X_train2, y_train)
    y_pred_rf = clf.predict(X_test)
    return accuracy_score(y_val, y_pred_rf)

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_rf, n_trials=20)
best_params_rf = study_rf.best_params_
print("\n Best Random Forest Hyperparameters:", best_params_rf)

Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler
import time

sc = RobustScaler()
X_train2 = sc.fit_transform(X_train)
X_test = sc.transform(X_val)

clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=45,
    min_samples_split=7,
    min_samples_leaf=3,
    random_state=1
)

start_time = time.time()
clf.fit(X_train2, y_train)
elapsed_time = time.time() - start_time
print(f"Training time: done {elapsed_time:.2f} seconds")

from sklearn.metrics import accuracy_score, classification_report
import numpy as np
y_pred = clf.predict(X_test)
prediction_time = time.time() - start_time
print(f"Prediction time: {prediction_time:.2f} seconds")

print("Accuracy Score: ", accuracy_score(y_val, y_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_pred))


feature_names = [f"feature {i}" for i in range(X_test.shape[1])]

importance = clf.feature_importances_

feature_names = X_train.columns if hasattr(X_train, 'columns') else [f"{i}" for i in range(X_train2.shape[1])]

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
})

sorted_df = feature_importance_df.sort_values('Importance', ascending=False)
sorted_df.to_csv('feature_importance_original.csv', index=False)


formatted_df = feature_importance_df.copy()


formatted_df['Feature'] = 'Feature ' + formatted_df['Feature'].astype(str)


formatted_df['Std'] = np.where(formatted_df['Importance'] > 0,
                             formatted_df['Importance'] * np.random.uniform(0.05, 0.15, size=len(formatted_df)),
                             0.0)


formatted_df.to_csv('ff.csv', index=False)


# CNN

In [ ]:
print("X_train_scaled shape:", X_train_scaled.shape)

# Reshape for 1D Conv layers, changes depending on what is classifying, if tyoe of device or device manufacturer or device
X_train_expanded = X_train_scaled.reshape(-1, 7, 1)
X_val_expanded = X_val_scaled.reshape(-1, 7, 1)

print("After reshaping:", X_train_expanded.shape)


best parameters

In [ ]:
import optuna
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import accuracy_score

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train.values)
X_val_scaled = sc.transform(X_val.values)

# Reshape for 1D Conv layers, changes depending on what is classifying, if tyoe of device or device manufacturer or device

X_train_expanded = X_train_scaled.reshape(-1, 6, 1)
X_val_expanded = X_val_scaled.reshape(-1, 6, 1)

y_train_bin = label_binarize(y_train, classes=np.unique(y_train))
y_val_bin = label_binarize(y_val, classes=np.unique(y_train))

def objective(trial):
    filters_1 = trial.suggest_int("filters_1", 16, 64, step=16)
    kernel_size_1 = trial.suggest_int("kernel_size_1", 2, 3)  # Smaller kernels for 1D
    use_second_conv = trial.suggest_categorical("use_second_conv", [True, False])
    filters_2 = trial.suggest_int("filters_2", 32, 128, step=16) if use_second_conv else None
    kernel_size_2 = trial.suggest_int("kernel_size_2", 2, 3) if use_second_conv else None
    dense_units = trial.suggest_int("dense_units", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)  # Fixed deprecation issue
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    model = Sequential([
        Input(shape=(6, 1)),
        Conv1D(filters_1, kernel_size=kernel_size_1, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2, padding="same")
    ])
    if use_second_conv:
        model.add(Conv1D(filters_2, kernel_size=kernel_size_2, activation='relu', padding='same'))
        model.add(MaxPooling1D(pool_size=2, padding="same"))

    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(y_train_bin.shape[1], activation='softmax'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train_expanded, y_train_bin, epochs=5, batch_size=batch_size, verbose=0, validation_data=(X_val_expanded, y_val_bin))

    y_pred = model.predict(X_val_expanded)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_val_classes = np.argmax(y_val_bin, axis=1)

    return accuracy_score(y_val_classes, y_pred_classes)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

print("\n Best CNN Hyperparameters:", study.best_params)


Training

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import numpy as np
import time

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train.values)
X_val_scaled = sc.transform(X_val.values)
# Reshape for 1D Conv layers, changes depending on what is classifying, if tyoe of device or device manufacturer or device
X_train_expanded = X_train_scaled.reshape(-1, 6, 1)
X_val_expanded = X_val_scaled.reshape(-1, 6, 1)

y_train_bin = label_binarize(y_train, classes=np.unique(y_train))
y_val_bin = label_binarize(y_val, classes=np.unique(y_train))

model = Sequential([
    Input(shape=(6, 1)),
    Conv1D(filters=48, kernel_size=2, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2, padding="same"),
    Conv1D(filters=48, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2, padding="same"),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(y_train_bin.shape[1], activation='softmax')
])
optimizer = Adam(learning_rate=0.0006013058752009507)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

start_time = time.time()
history = model.fit(X_train_expanded,
                   y_train_bin,
                   epochs=10,
                   batch_size=16,
                   validation_data=(X_val_expanded, y_val_bin))
elapsed_time = time.time() - start_time
print(f"Training time: {elapsed_time:.2f} seconds")
y_val_pred = model.predict(X_val_expanded)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)
y_val_true_classes = np.argmax(y_val_bin, axis=1)
accuracy = accuracy_score(y_val_true_classes, y_val_pred_classes)
f1 = f1_score(y_val_true_classes, y_val_pred_classes, average='weighted')
recall = recall_score(y_val_true_classes, y_val_pred_classes, average='weighted')
precision = precision_score(y_val_true_classes, y_val_pred_classes, average='weighted')

print("\nValidation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

# xgboost


Finding the best parameters for XGBoost

In [ ]:
import xgboost as xgb
import optuna
import time
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import cross_val_score

def optimize_xgboost(X_train, y_train):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int("n_estimators", 100, 500),
            'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3),
            'max_depth': trial.suggest_int("max_depth", 3, 15),
            'subsample': trial.suggest_float("subsample", 0.5, 1.0),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
            'gamma': trial.suggest_float("gamma", 0, 5),
            'reg_alpha': trial.suggest_float("reg_alpha", 0, 10),
            'reg_lambda': trial.suggest_float("reg_lambda", 0, 10)
        }

        model = xgb.XGBClassifier(**params, random_state=42, use_label_encoder=False, eval_metric="mlogloss")
        score = cross_val_score(model, X_train, y_train, cv=3, scoring="accuracy").mean()
        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=5)

    print("\n Best XGBoost Parameters Found:", study.best_params)
    return study.best_params

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
best_params = optimize_xgboost(X_train_scaled, y_train_encoded)


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import RobustScaler, LabelEncoder
import time
import numpy as np
import pandas as pd

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

clf = xgb.XGBClassifier(
    n_estimators=484,
    learning_rate=0.21,
    max_depth=11,
    gamma=0.47,
    subsample=0.84,
    colsample_bytree=0.64,
    reg_alpha=0.87,
    reg_lambda=7.55,
    random_state=42
)

start_time = time.time()
clf.fit(X_train_scaled, y_train_encoded)
training_time = time.time() - start_time
print(f"Training time: {training_time:.2f} seconds")

start_time = time.time()
y_pred_encoded = clf.predict(X_val_scaled)
prediction_time = time.time() - start_time
print(f"Prediction time: {prediction_time:.2f} seconds")

y_pred = le.inverse_transform(y_pred_encoded)

print("Accuracy Score: ", accuracy_score(y_val, y_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

importance = clf.feature_importances_

feature_names = X_train.columns if hasattr(X_train, 'columns') else [f"{i}" for i in range(X_train_scaled.shape[1])]

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
})

sorted_df = feature_importance_df.sort_values('Importance', ascending=False)
sorted_df.to_csv('feature_importance_original.csv', index=False)


formatted_df = feature_importance_df.copy()


formatted_df['Feature'] = 'Feature ' + formatted_df['Feature'].astype(str)


formatted_df['Std'] = np.where(formatted_df['Importance'] > 0,
                             formatted_df['Importance'] * np.random.uniform(0.05, 0.15, size=len(formatted_df)),
                             0.0)


formatted_df.to_csv('SNI-TYPe.csv', index=False)
